In [15]:
# Sample transactions
transactions = [
    ['milk', 'bread', 'butter'],
    ['milk', 'bread'],
    ['bread', 'butter'],
    ['milk', 'bread', 'butter', 'eggs'],
    ['milk', 'bread', 'eggs'],
    ['butter', 'eggs']
]

In [16]:
from itertools import combinations
from collections import Counter

def apriori(transactions, min_support, min_confidence):
    # Count item occurrences
    item_counts = Counter(item for transaction in transactions for item in transaction)
    
    # Generate frequent 1-itemsets
    n = len(transactions)
    frequent_itemsets = {1: {(item,): count for item, count in item_counts.items() if count >= min_support}}
    k = 2
    
    while frequent_itemsets[k-1]:
        candidates = set()
        for itemset1 in frequent_itemsets[k-1]:
            for itemset2 in frequent_itemsets[k-1]:
                union = tuple(sorted(set(itemset1) | set(itemset2)))
                if len(union) == k:
                    candidates.add(union)
        
        # Count candidate occurrences
        candidate_counts = Counter()
        for transaction in transactions:
            transaction_set = set(transaction)
            for candidate in candidates:
                if set(candidate).issubset(transaction_set):
                    candidate_counts[candidate] += 1
        
        # Filter frequent k-itemsets
        frequent_itemsets[k] = {itemset: count for itemset, count in candidate_counts.items() if count >= min_support}
        k += 1
    
    # Combine all frequent itemsets
    all_frequent_itemsets = {}
    for k_itemsets in frequent_itemsets.values():
        all_frequent_itemsets.update(k_itemsets)
    
    # Generate association rules
    rules = []
    for itemset, itemset_support in all_frequent_itemsets.items():
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = tuple(sorted(antecedent))
                    consequent = tuple(sorted(set(itemset) - set(antecedent)))
                    if antecedent in all_frequent_itemsets:
                        confidence = itemset_support / all_frequent_itemsets[antecedent]
                        if confidence >= min_confidence:
                            support = itemset_support / n
                            rules.append((antecedent, consequent, support, confidence))
    
    return all_frequent_itemsets, rules, n

In [17]:
def print_results(frequent_itemsets, rules, total_transactions):
    print("Frequent Itemsets:")
    for itemset, count in frequent_itemsets.items():
        support = count / total_transactions
        print(f"{itemset}: Support = {count}/{total_transactions} ({support:.4f})")
    
    print("\nAssociation Rules:")
    for antecedent, consequent, support, confidence in rules:
        print(f"{antecedent} -> {consequent}: Support = {support:.4f}, Confidence = {confidence:.4f}")



In [14]:
min_support = 2  # Minimum number of transactions
min_confidence = 0.6

frequent_itemsets, rules, total_transactions = apriori(transactions, min_support, min_confidence)
print_results(frequent_itemsets, rules, total_transactions)

Frequent Itemsets:
('milk',): Support = 4/6 (0.6667)
('bread',): Support = 5/6 (0.8333)
('butter',): Support = 4/6 (0.6667)
('eggs',): Support = 3/6 (0.5000)
('bread', 'butter'): Support = 3/6 (0.5000)
('butter', 'milk'): Support = 2/6 (0.3333)
('bread', 'milk'): Support = 4/6 (0.6667)
('butter', 'eggs'): Support = 2/6 (0.3333)
('eggs', 'milk'): Support = 2/6 (0.3333)
('bread', 'eggs'): Support = 2/6 (0.3333)
('bread', 'butter', 'milk'): Support = 2/6 (0.3333)
('bread', 'eggs', 'milk'): Support = 2/6 (0.3333)

Association Rules:
('bread',) -> ('butter',): Support = 0.5000, Confidence = 0.6000
('butter',) -> ('bread',): Support = 0.5000, Confidence = 0.7500
('bread',) -> ('milk',): Support = 0.6667, Confidence = 0.8000
('milk',) -> ('bread',): Support = 0.6667, Confidence = 1.0000
('eggs',) -> ('butter',): Support = 0.3333, Confidence = 0.6667
('eggs',) -> ('milk',): Support = 0.3333, Confidence = 0.6667
('eggs',) -> ('bread',): Support = 0.3333, Confidence = 0.6667
('bread', 'butter') 